In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "Vegetables_/train"
test_dir = "Vegetables_/test"
validation_dir = "Vegetables_/validation"

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest"
)

validation_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

# train_generator = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=(150,150),
#     batch_size = 32,
#     class_mode = "categorical"
# )

# test_generator = test_datagen.flow_from_directory(
#     test_dir,
#     target_size=(150,150),
#     batch_size = 32,
#     class_mode = "categorical"
# )

# validation_generator = validation_datagen.flow_from_directory(
#     validation_dir,
#     target_size=(150,150),
#     batch_size = 32,
#     class_mode = "categorical"
# )

In [20]:
from tensorflow.keras import datasets, layers, models


model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)), #nurodom filtrus ir ju dydziu dimensija
    layers.MaxPooling2D(2,2), #sumazinam dimensijas, islaikant svarbiausias savybes
    layers.Conv2D(64, (3,3),  activation='relu'), # antras konvoliucinis sluoksnis
    layers.MaxPooling2D(2,2), #sumazinam dimensijas, islaikant svarbiausias savybes
    layers.Conv2D(128, (3,3),  activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(), # plokstinam duomenis, paversdami is 3d i 1d
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(15, activation='softmax') # isejimo sluoksnis su vienu neuronu, grazinis tikimybe nuo 0 iki 1
])

In [21]:
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [22]:
print(train_generator.samples)
print(train_generator.batch_size)
print(validation_generator.samples)
print(validation_generator.batch_size)

15000
32
3000
32


In [23]:
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

model.save('Vegetables.h5')

Epoch 1/30


c:\Users\ITWORK\Desktop\Python\240319DMO492DIEmano\mokymai\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


468/468 ━━━━━━━━━━━━━━━━━━━━ 93s 197ms/step - accuracy: 0.2721 - loss: 2.1588 - val_accuracy: 0.6495 - val_loss: 1.0557
Epoch 2/30
468/468 ━━━━━━━━━━━━━━━━━━━━ 0s 80us/step - accuracy: 0.6250 - loss: 1.2102 - val_accuracy: 0.5833 - val_loss: 1.0642
Epoch 3/30
468/468 ━━━━━━━━━━━━━━━━━━━━ 92s 196ms/step - accuracy: 0.5945 - loss: 1.2273 - val_accuracy: 0.7604 - val_loss: 0.7417
Epoch 4/30
468/468 ━━━━━━━━━━━━━━━━━━━━ 0s 63us/step - accuracy: 0.7500 - loss: 0.6266 - val_accuracy: 0.6250 - val_loss: 1.0583
Epoch 5/30
468/468 ━━━━━━━━━━━━━━━━━━━━ 96s 204ms/step - accuracy: 0.6758 - loss: 0.9752 - val_accuracy: 0.7856 - val_loss: 0.6272
Epoch 6/30
468/468 ━━━━━━━━━━━━━━━━━━━━ 0s 70us/step - accuracy: 0.6250 - loss: 0.8956 - val_accuracy: 0.7083 - val_loss: 0.7142
Epoch 7/30
468/468 ━━━━━━━━━━━━━━━━━━━━ 98s 208ms/step - accuracy: 0.7355 - loss: 0.8124 - val_accuracy: 0.8233 - val_loss: 0.5331
Epoch 8/30
468/468 ━━━━━━━━━━━━━━━━━━━━ 0s 67us/step - accuracy: 0.7812 - loss: 0.7977 - val_accurac

In [24]:
model.save('Vegetables.h5')

In [36]:
import numpy as np
from PIL import Image

def preprocess_image(image_path):
    img = Image.open(image_path)
    if image_path.endswith(".png"):
        im = Image.open(image_path)
        img = im.convert('RGB')
    img = img.resize((150,150))
    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

class_indices = train_generator.class_indices
class_labels = {value: key for key, value in class_indices.items()}


def predict_vegetable(path_to_img):
    img_array = preprocess_image(path_to_img)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return class_labels[predicted_class[0]]

# print(predict_vegetable('morka.png'))
# print(predict_vegetable('morka2.jpg'))
# # print(predict_vegetable('pomidoras.jpg'))
# print(predict_vegetable('pomidoras2.jpg'))



In [37]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(test_loss)
print(test_accuracy)

 1/93 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 1.0000 - loss: 0.0759

c:\Users\ITWORK\Desktop\Python\240319DMO492DIEmano\mokymai\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


93/93 ━━━━━━━━━━━━━━━━━━━━ 14s 155ms/step - accuracy: 0.9526 - loss: 0.1719
0.14436200261116028
0.9596773982048035
